In [6]:
import pandas as pd
import numpy as np
import json
import xlsxwriter
import glob
import xlrd
from datetime import datetime
pd.options.mode.chained_assignment = None

Use Python 3.9.15

In [7]:
df_1 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/1. TrEquity 2008 sd 2010.csv')
df_2 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/2. TrEquity 2011 sd 2015.csv')
df_3 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/3. TrEquity 2016 sd 2020.csv')
df_4 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/4. TrEquity 2021 sd 2023.csv')
df_5 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/5. TrEquity 01-01-2023 sd 15-10-2024.csv')
df_6 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/6. TrEquity 16-10-2024 sd 16-12-2024.csv')
df_7 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/7. TrEquity 17-12-2024 sd 08-01-2025.csv')
df_all = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7])

/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/4019134917.py:3: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_3 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/3. TrEquity 2016 sd 2020.csv')
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/4019134917.py:4: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_4 = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/TrEquity/4. TrEquity 2021 sd 2023.csv')


In [8]:
#Jika sudah run pembuatan database Laporan Keuangan, mulai dari sini!! Jika belum, buka Scraping_Stockbit.ipynb
income_statement = pd.read_excel('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Income Statement.xlsx',index_col=0)
income_statement['Kode + Quarter'] = income_statement['Kode'] + ' ' + income_statement['Quarter'].astype(str)
# income_statement = income_statement.drop(columns=['Arus Kas Dari Aktivitas Operasi','Arus Kas Dari Aktivitas Investasi',
                                                #   'Arus Kas Dari Aktivitas Pendanaan','Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'])

balance_sheet = pd.read_excel('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Balance Sheet.xlsx',index_col=0)
balance_sheet['Kode + Quarter'] = balance_sheet['Kode'] + ' ' + balance_sheet['Quarter'].astype(str)

cash_flow = pd.read_excel('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Cash Flow.xlsx',index_col=0)
cash_flow['Kode + Quarter'] = cash_flow['Kode'] + ' ' + cash_flow['Quarter'].astype(str)

In [9]:
# Tambah Quarter di lapkeu jika data di TrEquity melebihi tanggal di lapkeu. Tujuannya supaya begitu di-join, data tidak terpotong di quarter terakhir lapkeu.
# Contoh: Quarter terakhir lapkeu adalah Q32024 sedangkan tanggal di TrEquity sampai 10-Okt-2024, berarti tambah Quarter Q4 2024
add_quarter = ['Q1 2025','Q4 2024'] # --> Harus urut jika lebih dari satu!!!! Contoh : ['Q1 2025','Q4 2024']

s_list = []
for s in list(income_statement['Kode'].unique()):
    data_s =income_statement[income_statement['Kode']==s]
    data_q = pd.DataFrame({'Kode':s,'Quarter':add_quarter})
    data_q = data_q.assign(**pd.DataFrame(columns=income_statement.columns[2:])) 
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_s = pd.concat([data_q,data_s])
    s_list.append(data_s)
income_statement = pd.concat(s_list)

s_list = []
for s in list(balance_sheet['Kode'].unique()):
    data_s =balance_sheet[balance_sheet['Kode']==s]
    data_q = pd.DataFrame({'Kode':s,'Quarter':add_quarter})
    data_q = data_q.assign(**pd.DataFrame(columns=balance_sheet.columns[2:])) 
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_s = pd.concat([data_q,data_s])
    s_list.append(data_s)
balance_sheet = pd.concat(s_list)

s_list = []
for s in list(cash_flow['Kode'].unique()):
    data_s =cash_flow[cash_flow['Kode']==s]
    data_q = pd.DataFrame({'Kode':s,'Quarter':add_quarter})
    data_q = data_q.assign(**pd.DataFrame(columns=cash_flow.columns[2:])) 
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_s = pd.concat([data_q,data_s])
    s_list.append(data_s)
cash_flow = pd.concat(s_list)

/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/2552801110.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_s = pd.concat([data_q,data_s])
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/2552801110.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_s = pd.concat([data_q,data_s])
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/2552801110.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is 

In [10]:
lapkeu_sb = income_statement.copy()
lapkeu_sb = lapkeu_sb.merge(balance_sheet.drop(['Kode','Quarter'],axis=1), how='outer', on='Kode + Quarter')
lapkeu_sb = lapkeu_sb.merge(cash_flow.drop(['Kode','Quarter'],axis=1), how='outer', on='Kode + Quarter')
lapkeu_sb['Kode'] = [c[:4] for c in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Quarter'] = [q[5:7] for q in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Year'] = [y[8:] for y in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Quarter Year'] = lapkeu_sb['Quarter'].astype('str') + lapkeu_sb['Year'].astype('str')
lapkeu_sb.insert(2, 'Year', lapkeu_sb.pop('Year'))
lapkeu_sb.insert(3, 'Quarter Year', lapkeu_sb.pop('Quarter Year'))
lapkeu_sb = lapkeu_sb.drop(columns=['Kode + Quarter'])

In [11]:
#Membuat dictionary berisi Quarter Number (ingat harus urut!! cek dulu di excel)
quarter_num = {}
keys = list(lapkeu_sb['Quarter Year'].unique())
values = range(len(list(lapkeu_sb['Quarter Year'].unique())))
index_ = [i for i in range(len(list(lapkeu_sb['Quarter Year'].unique())))]
for eu,u in enumerate(keys):
    quarter_num[u] = [values[eu]+1]
#pd.DataFrame(quarter_num).T.to_excel('dict.xlsx')

#Buat Quarter Number
lapkeu_sb['Quarter Number'] = [quarter_num[x][0] for x in lapkeu_sb['Quarter Year']]

In [12]:
for i in range(len(lapkeu_sb)):
    if i < (len(lapkeu_sb)-4):
        if (lapkeu_sb.loc[i,'Kode'] == lapkeu_sb.loc[i+3,'Kode']) and ((lapkeu_sb.loc[i+1,'Quarter Number']-lapkeu_sb.loc[i,'Quarter Number']) == 1) and  ((lapkeu_sb.loc[i+2,'Quarter Number']-lapkeu_sb.loc[i+1,'Quarter Number']) == 1) and ((lapkeu_sb.loc[i+3,'Quarter Number']-lapkeu_sb.loc[i+2,'Quarter Number']) == 1):
            if (lapkeu_sb.loc[i+1,'Quarter Number'] == ((lapkeu_sb.loc[i,'Quarter Number']) + 1)) and (lapkeu_sb.loc[i+2,'Quarter Number'] == ((lapkeu_sb.loc[i+1,'Quarter Number']) + 1)) and (lapkeu_sb.loc[i+3,'Quarter Number'] == ((lapkeu_sb.loc[i+2,'Quarter Number']) + 1)):
                lapkeu_sb.loc[i,'Total Pendapatan (annualized)'] = lapkeu_sb.loc[i,'Total Pendapatan'] + lapkeu_sb.loc[i+1,'Total Pendapatan'] + lapkeu_sb.loc[i+2,'Total Pendapatan'] + lapkeu_sb.loc[i+3,'Total Pendapatan']
                lapkeu_sb.loc[i,'Laba Kotor (annualized)'] = lapkeu_sb.loc[i,'Laba Kotor'] + lapkeu_sb.loc[i+1,'Laba Kotor'] + lapkeu_sb.loc[i+2,'Laba Kotor'] + lapkeu_sb.loc[i+3,'Laba Kotor']
                lapkeu_sb.loc[i,'Laba Usaha (annualized)'] = lapkeu_sb.loc[i,'Laba Usaha'] + lapkeu_sb.loc[i+1,'Laba Usaha'] + lapkeu_sb.loc[i+2,'Laba Usaha'] + lapkeu_sb.loc[i+3,'Laba Usaha']
                lapkeu_sb.loc[i,'Laba Sebelum Pajak (annualized)'] = lapkeu_sb.loc[i,'Laba Sebelum Pajak'] + lapkeu_sb.loc[i+1,'Laba Sebelum Pajak'] + lapkeu_sb.loc[i+2,'Laba Sebelum Pajak'] + lapkeu_sb.loc[i+3,'Laba Sebelum Pajak']
                lapkeu_sb.loc[i,'Laba Bersih Tahun Berjalan (annualized)'] = lapkeu_sb.loc[i,'Laba Bersih Tahun Berjalan'] + lapkeu_sb.loc[i+1,'Laba Bersih Tahun Berjalan'] + lapkeu_sb.loc[i+2,'Laba Bersih Tahun Berjalan'] + lapkeu_sb.loc[i+3,'Laba Bersih Tahun Berjalan']
                lapkeu_sb.loc[i,'Jumlah Laba Komprehensif (annualized)'] = lapkeu_sb.loc[i,'Jumlah Laba Komprehensif'] + lapkeu_sb.loc[i+1,'Jumlah Laba Komprehensif'] + lapkeu_sb.loc[i+2,'Jumlah Laba Komprehensif'] + lapkeu_sb.loc[i+3,'Jumlah Laba Komprehensif']
                lapkeu_sb.loc[i,'Laba Bersih Yang Dapat Diatribusikan Kepada (annualized)'] = lapkeu_sb.loc[i,'Laba Bersih Yang Dapat Diatribusikan Kepada'] + lapkeu_sb.loc[i+1,'Laba Bersih Yang Dapat Diatribusikan Kepada'] + lapkeu_sb.loc[i+2,'Laba Bersih Yang Dapat Diatribusikan Kepada'] + lapkeu_sb.loc[i+3,'Laba Bersih Yang Dapat Diatribusikan Kepada']
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Operasi (annualized)'] = lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Operasi'] + lapkeu_sb.loc[i+1,'Arus Kas Dari Aktivitas Operasi'] + lapkeu_sb.loc[i+2,'Arus Kas Dari Aktivitas Operasi'] + lapkeu_sb.loc[i+3,'Arus Kas Dari Aktivitas Operasi']
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Investasi (annualized)'] = lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Investasi'] + lapkeu_sb.loc[i+1,'Arus Kas Dari Aktivitas Investasi'] + lapkeu_sb.loc[i+2,'Arus Kas Dari Aktivitas Investasi'] + lapkeu_sb.loc[i+3,'Arus Kas Dari Aktivitas Investasi']
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Pendanaan (annualized)'] = lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Pendanaan'] + lapkeu_sb.loc[i+1,'Arus Kas Dari Aktivitas Pendanaan'] + lapkeu_sb.loc[i+2,'Arus Kas Dari Aktivitas Pendanaan'] + lapkeu_sb.loc[i+3,'Arus Kas Dari Aktivitas Pendanaan']
                lapkeu_sb.loc[i,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas (annualized)'] = lapkeu_sb.loc[i,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'] + lapkeu_sb.loc[i+1,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'] + lapkeu_sb.loc[i+2,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'] + lapkeu_sb.loc[i+3,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas']
            else:
                lapkeu_sb.loc[i,'Total Pendapatan (annualized)'] = ''
                lapkeu_sb.loc[i,'Laba Kotor (annualized)'] = ''
                lapkeu_sb.loc[i,'Laba Usaha (annualized)'] = '' 
                lapkeu_sb.loc[i,'Laba Sebelum Pajak (annualized)'] = ''
                lapkeu_sb.loc[i,'Laba Bersih Tahun Berjalan (annualized)'] = ''
                lapkeu_sb.loc[i,'Jumlah Laba Komprehensif (annualized)'] = ''
                lapkeu_sb.loc[i,'Laba Bersih Yang Dapat Diatribusikan Kepada (annualized)'] = ''
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Operasi (annualized)'] = ''
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Investasi (annualized)'] = ''
                lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Pendanaan (annualized)'] = ''
                lapkeu_sb.loc[i,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas (annualized)'] = ''
        else:
            lapkeu_sb.loc[i,'Total Pendapatan (annualized)'] = ''
            lapkeu_sb.loc[i,'Laba Kotor (annualized)'] = ''
            lapkeu_sb.loc[i,'Laba Usaha (annualized)'] = '' 
            lapkeu_sb.loc[i,'Laba Sebelum Pajak (annualized)'] = ''
            lapkeu_sb.loc[i,'Laba Bersih Tahun Berjalan (annualized)'] = '' 
            lapkeu_sb.loc[i,'Jumlah Laba Komprehensif (annualized)'] = '' 
            lapkeu_sb.loc[i,'Laba Bersih Yang Dapat Diatribusikan Kepada (annualized)'] = '' 
            lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Operasi (annualized)'] = '' 
            lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Investasi (annualized)'] = '' 
            lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Pendanaan (annualized)'] = '' 
            lapkeu_sb.loc[i,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas (annualized)'] = '' 
    else:
        lapkeu_sb.loc[i,'Total Pendapatan (annualized)'] = ''
        lapkeu_sb.loc[i,'Laba Kotor (annualized)'] = ''
        lapkeu_sb.loc[i,'Laba Usaha (annualized)'] = '' 
        lapkeu_sb.loc[i,'Laba Sebelum Pajak (annualized)'] = ''
        lapkeu_sb.loc[i,'Laba Bersih Tahun Berjalan (annualized)'] = ''
        lapkeu_sb.loc[i,'Jumlah Laba Komprehensif (annualized)'] = ''
        lapkeu_sb.loc[i,'Laba Bersih Yang Dapat Diatribusikan Kepada (annualized)'] = ''
        lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Operasi (annualized)'] = ''
        lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Investasi (annualized)'] = ''
        lapkeu_sb.loc[i,'Arus Kas Dari Aktivitas Pendanaan (annualized)'] = ''
        lapkeu_sb.loc[i,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas (annualized)'] = ''
lapkeu_sb = lapkeu_sb.drop(columns=['Total Pendapatan','Laba Kotor','Laba Usaha','Laba Sebelum Pajak','Laba Bersih Tahun Berjalan',
                        'Jumlah Laba Komprehensif','Laba Bersih Yang Dapat Diatribusikan Kepada','Arus Kas Dari Aktivitas Operasi',
                        'Arus Kas Dari Aktivitas Investasi','Arus Kas Dari Aktivitas Pendanaan','Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'])

#Buat agar tiap saham punya jumlah Quarter Year yg sama, supaya begitu join dgn price data tidak ada saham yg skip
quarter_year_list = list(lapkeu_sb['Quarter Year'].unique())
lapkeu_sb_i_list = []
for i in lapkeu_sb['Kode'].unique():
    lapkeu_sb_i = lapkeu_sb[lapkeu_sb['Kode']==i]
    quarter_year_list_i = list(lapkeu_sb_i['Quarter Year'])
    quarter_year_list_not_in_i = [x for x in quarter_year_list if x not in quarter_year_list_i]
    df_lapkeu_sb_i = pd.DataFrame(index=list(range(len(quarter_year_list_not_in_i))), columns=lapkeu_sb_i.columns)
    df_lapkeu_sb_i['Quarter Year'] = quarter_year_list_not_in_i
    df_lapkeu_sb_i['Kode'] = lapkeu_sb_i['Kode'].iloc[0]
    df_lapkeu_sb_i['Quarter'] = df_lapkeu_sb_i['Quarter Year'].str[:2]
    df_lapkeu_sb_i['Year'] = df_lapkeu_sb_i['Quarter Year'].str[2:]
    lapkeu_sb_i = pd.concat([lapkeu_sb_i,df_lapkeu_sb_i])
    lapkeu_sb_i = lapkeu_sb_i.sort_values(by=['Year','Quarter'],ascending=False).reset_index(drop=True)
    lapkeu_sb_i_list.append(lapkeu_sb_i)
lapkeu_sb = pd.concat(lapkeu_sb_i_list)
lapkeu_sb = lapkeu_sb.reset_index(drop=True)

#Buat Date From dan Date To
for i in range(len(lapkeu_sb)):
    if lapkeu_sb.loc[i,'Quarter'] == 'Q1':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-01-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-03-31'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q2':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-04-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-06-30'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q3':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-07-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-09-30'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q4':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-10-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-12-31'
    else:
        lapkeu_sb.loc[i,'Date From'] = '' 
        lapkeu_sb.loc[i,'Date To'] = ''

lapkeu_sb_annualized = lapkeu_sb.copy()
lapkeu_sb_annualized.columns = lapkeu_sb_annualized.columns.str.replace(' (annualized)','',regex=False) #semua data selain balance sheet sudah di-anualizedkan


/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/4191518254.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  lapkeu_sb.loc[i,'Total Pendapatan (annualized)'] = ''
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/4191518254.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  lapkeu_sb.loc[i,'Laba Kotor (annualized)'] = ''
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/4191518254.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatibl

In [13]:
#Augment Quarter Year yang belum lengkap untuk saham2 yg lapkeu-nya belum dirilis sesuai kuartal terakhir (misal MDKA rilis terakhir Q2 2024, sedangkan saham lain sudah Q3 2024)
#Tujuannya agar begitu di-join, data terakhir lebih lengkap karena data terakhir nantinya akan di lag-2
date_from_list = list(lapkeu_sb_annualized['Date From'].unique())
nearest_date_from = lapkeu_sb_annualized['Date From'].max()
nearest_date_from_ = datetime.strptime(nearest_date_from, '%Y-%m-%d').date()
date_to_list = list(lapkeu_sb_annualized['Date To'].unique())
nearest_date_to = lapkeu_sb_annualized['Date To'].max()
quarter_dict = {'03':'Q1','06':'Q2','09':'Q3','12':'Q4'}
lapkeu_sb_annualized_c_merged_list = []
for c in lapkeu_sb_annualized['Kode'].unique():
    lapkeu_sb_annualized_c = lapkeu_sb_annualized[lapkeu_sb_annualized['Kode']==c]
    last_date_from = lapkeu_sb_annualized_c['Date From'].iloc[0]
    last_date_from_ = datetime.strptime(last_date_from, '%Y-%m-%d').date()
    if last_date_from_ < nearest_date_from_:
        new_date_from_list = list(filter(lambda d: d > last_date_from, date_from_list))
        last_date_to = lapkeu_sb_annualized_c['Date To'].iloc[0]
        new_date_to_list = list(filter(lambda d: d > last_date_to, date_to_list))
        new_lapkeu_df = pd.DataFrame({'Date From':new_date_from_list,'Date To':new_date_to_list})
        new_lapkeu_df['Kode'] = c
        new_lapkeu_df['Year'] = new_lapkeu_df['Date To'].str[:4]
        new_lapkeu_df['Quarter'] = new_lapkeu_df['Date To'].str[5:7]
        new_lapkeu_df['Quarter'] = new_lapkeu_df['Quarter'].map(quarter_dict)
        new_lapkeu_df['Quarter Year'] = new_lapkeu_df['Quarter'].astype('str') + new_lapkeu_df['Year'].astype('str')
        lapkeu_sb_annualized_c_merged = lapkeu_sb_annualized_c.merge(new_lapkeu_df,on=['Kode','Quarter','Year','Quarter Year','Date From','Date To'],how='outer')
        lapkeu_sb_annualized_c_merged = lapkeu_sb_annualized_c_merged.sort_values(by='Date From',ascending=False).reset_index(drop=True)
        lapkeu_sb_annualized_c_merged_list.append(lapkeu_sb_annualized_c_merged)
    else: 
        lapkeu_sb_annualized_c_merged_list.append(lapkeu_sb_annualized_c)
lapkeu_sb_annualized = pd.concat(lapkeu_sb_annualized_c_merged_list)
lapkeu_sb_annualized['Quarter Number'] = [quarter_num[x][0] for x in lapkeu_sb_annualized['Quarter Year']]

In [14]:
lapkeu_sb_annualized

,Kode,Quarter,Year,Quarter Year,Aset Lancar,Aset Tidak Lancar,Aset,Liabilitas Jangka Pendek,Liabilitas Jangka Panjang,Liabilitas,...,Laba Sebelum Pajak,Laba Bersih Tahun Berjalan,Jumlah Laba Komprehensif,Laba Bersih Yang Dapat Diatribusikan Kepada,Arus Kas Dari Aktivitas Operasi,Arus Kas Dari Aktivitas Investasi,Arus Kas Dari Aktivitas Pendanaan,Kenaikan (Penurunan) Bersih Kas Dan Setara Kas,Date From,Date To
0,AADI,Q1,2025,Q12025,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-01,2025-03-31
1,AADI,Q4,2024,Q42024,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,2024-10-01,2024-12-31
2,AADI,Q3,2024,Q32024,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-01,2024-09-30
3,AADI,Q2,2024,Q22024,3.075501e+13,5.831421e+13,8.906922e+13,2.491662e+13,1.954506e+13,4.446168e+13,...,103645829873000.0,71384271977000.0,72531995994000.0,71384271977000.0,85974386242000.0,9217294375000.0,-72213880749000.0,22977799868000.0,2024-04-01,2024-06-30
4,AADI,Q1,2024,Q12024,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70147,ZYRX,Q4,2007,Q42007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-10-01,2007-12-31
70148,ZYRX,Q3,2007,Q32007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-07-01,2007-09-30
70149,ZYRX,Q2,2007,Q22007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-04-01,2007-06-30
70150,ZYRX,Q1,2007,Q12007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-01,2007-03-31


In [15]:
list_lapkeu_lag = []
for i in set(list(lapkeu_sb_annualized['Kode'])):
    lapkeu_i = lapkeu_sb_annualized[lapkeu_sb_annualized['Kode']==i]
    #Buat lag 2
    lapkeu_i_lag_2 = lapkeu_i[['Kode']]
    lapkeu_i_lag_2.loc[:,'Aset Lancar Lag 2'] = lapkeu_i['Aset Lancar'].values
    lapkeu_i_lag_2.loc[:,'Aset Tidak Lancar Lag 2'] = lapkeu_i['Aset Tidak Lancar'].values
    lapkeu_i_lag_2.loc[:,'Aset Lag 2'] = lapkeu_i['Aset'].values
    lapkeu_i_lag_2.loc[:,'Liabilitas Jangka Pendek Lag 2'] = lapkeu_i['Liabilitas Jangka Pendek'].values
    lapkeu_i_lag_2.loc[:,'Liabilitas Jangka Panjang Lag 2'] = lapkeu_i['Liabilitas Jangka Panjang'].values
    lapkeu_i_lag_2.loc[:,'Liabilitas Lag 2'] = lapkeu_i['Liabilitas'].values
    lapkeu_i_lag_2.loc[:,'Ekuitas Lag 2'] = lapkeu_i['Ekuitas'].values
    lapkeu_i_lag_2.loc[:,'Liabilitas Dan Ekuitas Lag 2'] = lapkeu_i['Liabilitas Dan Ekuitas'].values
    lapkeu_i_lag_2.loc[:,'Kas Dan Setara Kas Lag 2'] = lapkeu_i['Kas Dan Setara Kas'].values
    lapkeu_i_lag_2.loc[:,'Piutang Premi Lag 2'] = lapkeu_i['Piutang Premi'].values
    lapkeu_i_lag_2.loc[:,'Piutang Reasuransi Lag 2'] = lapkeu_i['Piutang Reasuransi'].values
    lapkeu_i_lag_2.loc[:,'Penempatan Pada Bank Indonesia Dan Bank Lain Lag 2'] = lapkeu_i['Penempatan Pada Bank Indonesia Dan Bank Lain'].values
    lapkeu_i_lag_2.loc[:,'Pinjaman Yang Diberikan Lag 2'] = lapkeu_i['Pinjaman Yang Diberikan'].values
    lapkeu_i_lag_2.loc[:,'Aset Tetap Lag 2'] = lapkeu_i['Aset Tetap'].values
    lapkeu_i_lag_2.loc[:,'Piutang Sewa Pembiayaan Lag 2'] = lapkeu_i['Piutang Sewa Pembiayaan'].values
    lapkeu_i_lag_2.loc[:,'Piutang Pembiayaan Konsumen Lag 2'] = lapkeu_i['Piutang Pembiayaan Konsumen'].values
    lapkeu_i_lag_2.loc[:,'Piutang Nasabah Lag 2'] = lapkeu_i['Piutang Nasabah'].values
    lapkeu_i_lag_2.loc[:,'Piutang Lain-Lain Lag 2'] = lapkeu_i['Piutang Lain-Lain'].values
    lapkeu_i_lag_2.loc[:,'Portofolio Efek Lag 2'] = lapkeu_i['Portofolio Efek'].values
    lapkeu_i_lag_2.loc[:,'Aset Tak Berwujud Lag 2'] = lapkeu_i['Aset Tak Berwujud'].values
    lapkeu_i_lag_2.loc[:,'Investasi Jangka Pendek Lag 2'] = lapkeu_i['Investasi Jangka Pendek'].values
    lapkeu_i_lag_2.loc[:,'Aset Reasuransi Lag 2'] = lapkeu_i['Aset Reasuransi'].values
    lapkeu_i_lag_2.loc[:,'Piutang Usaha Lag 2'] = lapkeu_i['Piutang Usaha'].values
    lapkeu_i_lag_2.loc[:,'Piutang Premi Dan Reasuransi Lag 2'] = lapkeu_i['Piutang Premi Dan Reasuransi'].values
    lapkeu_i_lag_2.loc[:,'Investasi Jangka Panjang Lag 2'] = lapkeu_i['Investasi Jangka Panjang'].values
    lapkeu_i_lag_2.loc[:,'Goodwill Lag 2'] = lapkeu_i['Goodwill'].values
    lapkeu_i_lag_2.loc[:,'Piutang Pembiayaan Lag 2'] = lapkeu_i['Piutang Pembiayaan'].values
    lapkeu_i_lag_2.loc[:,'Obligasi Pemerintah Lag 2'] = lapkeu_i['Obligasi Pemerintah'].values
    lapkeu_i_lag_2.loc[:,'Investasi Pada Entitas Asosiasi Lag 2'] = lapkeu_i['Investasi Pada Entitas Asosiasi'].values
    lapkeu_i_lag_2.loc[:,'Deposito Berjangka Lag 2'] = lapkeu_i['Deposito Berjangka'].values
    lapkeu_i_lag_2.loc[:,'Efek Tersedia Untuk Dijual Lag 2'] = lapkeu_i['Efek Tersedia Untuk Dijual'].values
    lapkeu_i_lag_2.loc[:,'Total Pendapatan Lag 2'] = lapkeu_i['Total Pendapatan'].values
    lapkeu_i_lag_2.loc[:,'Laba Kotor Lag 2'] = lapkeu_i['Laba Kotor'].values
    lapkeu_i_lag_2.loc[:,'Laba Usaha Lag 2'] = lapkeu_i['Laba Usaha'].values
    lapkeu_i_lag_2.loc[:,'Laba Sebelum Pajak Lag 2'] = lapkeu_i['Laba Sebelum Pajak'].values
    lapkeu_i_lag_2.loc[:,'Laba Bersih Tahun Berjalan Lag 2'] = lapkeu_i['Laba Bersih Tahun Berjalan'].values
    lapkeu_i_lag_2.loc[:,'Jumlah Laba Komprehensif Lag 2'] = lapkeu_i['Jumlah Laba Komprehensif'].values
    lapkeu_i_lag_2.loc[:,'Laba Bersih Yang Dapat Diatribusikan Kepada Lag 2'] = lapkeu_i['Laba Bersih Yang Dapat Diatribusikan Kepada'].values
    lapkeu_i_lag_2.loc[:,'Arus Kas Dari Aktivitas Operasi Lag 2'] = lapkeu_i['Arus Kas Dari Aktivitas Operasi'].values
    lapkeu_i_lag_2.loc[:,'Arus Kas Dari Aktivitas Investasi Lag 2'] = lapkeu_i['Arus Kas Dari Aktivitas Investasi'].values
    lapkeu_i_lag_2.loc[:,'Arus Kas Dari Aktivitas Pendanaan Lag 2'] = lapkeu_i['Arus Kas Dari Aktivitas Pendanaan'].values
    lapkeu_i_lag_2.loc[:,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 2'] = lapkeu_i['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'].values
    lapkeu_i_lag_2.loc[:,'Quarter Number'] = lapkeu_i['Quarter Number'] - 2 #Buat lag Quarter Number untuk join
    lapkeu_i_lag_2.drop(columns=['Kode'],inplace=True)
    lapkeu_i = lapkeu_i.merge(lapkeu_i_lag_2, how='outer', on='Quarter Number')
    #Buat lag 6 untuk menghitung YoY growth
    lapkeu_i_lag_6 = lapkeu_i[['Kode']]
    lapkeu_i_lag_6.loc[:,'Aset Lancar Lag 6'] = lapkeu_i['Aset Lancar'].values
    lapkeu_i_lag_6.loc[:,'Aset Tidak Lancar Lag 6'] = lapkeu_i['Aset Tidak Lancar'].values
    lapkeu_i_lag_6.loc[:,'Aset Lag 6'] = lapkeu_i['Aset'].values
    lapkeu_i_lag_6.loc[:,'Liabilitas Jangka Pendek Lag 6'] = lapkeu_i['Liabilitas Jangka Pendek'].values
    lapkeu_i_lag_6.loc[:,'Liabilitas Jangka Panjang Lag 6'] = lapkeu_i['Liabilitas Jangka Panjang'].values
    lapkeu_i_lag_6.loc[:,'Liabilitas Lag 6'] = lapkeu_i['Liabilitas'].values
    lapkeu_i_lag_6.loc[:,'Ekuitas Lag 6'] = lapkeu_i['Ekuitas'].values
    lapkeu_i_lag_6.loc[:,'Liabilitas Dan Ekuitas Lag 6'] = lapkeu_i['Liabilitas Dan Ekuitas'].values
    lapkeu_i_lag_6.loc[:,'Kas Dan Setara Kas Lag 6'] = lapkeu_i['Kas Dan Setara Kas'].values
    lapkeu_i_lag_6.loc[:,'Piutang Premi Lag 6'] = lapkeu_i['Piutang Premi'].values
    lapkeu_i_lag_6.loc[:,'Piutang Reasuransi Lag 6'] = lapkeu_i['Piutang Reasuransi'].values
    lapkeu_i_lag_6.loc[:,'Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6'] = lapkeu_i['Penempatan Pada Bank Indonesia Dan Bank Lain'].values
    lapkeu_i_lag_6.loc[:,'Pinjaman Yang Diberikan Lag 6'] = lapkeu_i['Pinjaman Yang Diberikan'].values
    lapkeu_i_lag_6.loc[:,'Aset Tetap Lag 6'] = lapkeu_i['Aset Tetap'].values
    lapkeu_i_lag_6.loc[:,'Piutang Sewa Pembiayaan Lag 6'] = lapkeu_i['Piutang Sewa Pembiayaan'].values
    lapkeu_i_lag_6.loc[:,'Piutang Pembiayaan Konsumen Lag 6'] = lapkeu_i['Piutang Pembiayaan Konsumen'].values
    lapkeu_i_lag_6.loc[:,'Piutang Nasabah Lag 6'] = lapkeu_i['Piutang Nasabah'].values
    lapkeu_i_lag_6.loc[:,'Piutang Lain-Lain Lag 6'] = lapkeu_i['Piutang Lain-Lain'].values
    lapkeu_i_lag_6.loc[:,'Portofolio Efek Lag 6'] = lapkeu_i['Portofolio Efek'].values
    lapkeu_i_lag_6.loc[:,'Aset Tak Berwujud Lag 6'] = lapkeu_i['Aset Tak Berwujud'].values
    lapkeu_i_lag_6.loc[:,'Investasi Jangka Pendek Lag 6'] = lapkeu_i['Investasi Jangka Pendek'].values
    lapkeu_i_lag_6.loc[:,'Aset Reasuransi Lag 6'] = lapkeu_i['Aset Reasuransi'].values
    lapkeu_i_lag_6.loc[:,'Piutang Usaha Lag 6'] = lapkeu_i['Piutang Usaha'].values
    lapkeu_i_lag_6.loc[:,'Piutang Premi Dan Reasuransi Lag 6'] = lapkeu_i['Piutang Premi Dan Reasuransi'].values
    lapkeu_i_lag_6.loc[:,'Investasi Jangka Panjang Lag 6'] = lapkeu_i['Investasi Jangka Panjang'].values
    lapkeu_i_lag_6.loc[:,'Goodwill Lag 6'] = lapkeu_i['Goodwill'].values
    lapkeu_i_lag_6.loc[:,'Piutang Pembiayaan Lag 6'] = lapkeu_i['Piutang Pembiayaan'].values
    lapkeu_i_lag_6.loc[:,'Obligasi Pemerintah Lag 6'] = lapkeu_i['Obligasi Pemerintah'].values
    lapkeu_i_lag_6.loc[:,'Investasi Pada Entitas Asosiasi Lag 6'] = lapkeu_i['Investasi Pada Entitas Asosiasi'].values
    lapkeu_i_lag_6.loc[:,'Deposito Berjangka Lag 6'] = lapkeu_i['Deposito Berjangka'].values
    lapkeu_i_lag_6.loc[:,'Efek Tersedia Untuk Dijual Lag 6'] = lapkeu_i['Efek Tersedia Untuk Dijual'].values
    lapkeu_i_lag_6.loc[:,'Total Pendapatan Lag 6'] = lapkeu_i['Total Pendapatan'].values
    lapkeu_i_lag_6.loc[:,'Laba Kotor Lag 6'] = lapkeu_i['Laba Kotor'].values
    lapkeu_i_lag_6.loc[:,'Laba Usaha Lag 6'] = lapkeu_i['Laba Usaha'].values
    lapkeu_i_lag_6.loc[:,'Laba Sebelum Pajak Lag 6'] = lapkeu_i['Laba Sebelum Pajak'].values
    lapkeu_i_lag_6.loc[:,'Laba Bersih Tahun Berjalan Lag 6'] = lapkeu_i['Laba Bersih Tahun Berjalan'].values
    lapkeu_i_lag_6.loc[:,'Jumlah Laba Komprehensif Lag 6'] = lapkeu_i['Jumlah Laba Komprehensif'].values
    lapkeu_i_lag_6.loc[:,'Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6'] = lapkeu_i['Laba Bersih Yang Dapat Diatribusikan Kepada'].values
    lapkeu_i_lag_6.loc[:,'Arus Kas Dari Aktivitas Operasi Lag 6'] = lapkeu_i['Arus Kas Dari Aktivitas Operasi'].values
    lapkeu_i_lag_6.loc[:,'Arus Kas Dari Aktivitas Investasi Lag 6'] = lapkeu_i['Arus Kas Dari Aktivitas Investasi'].values
    lapkeu_i_lag_6.loc[:,'Arus Kas Dari Aktivitas Pendanaan Lag 6'] = lapkeu_i['Arus Kas Dari Aktivitas Pendanaan'].values
    lapkeu_i_lag_6.loc[:,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'] = lapkeu_i['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'].values
    lapkeu_i_lag_6.loc[:,'Quarter Number'] = lapkeu_i.loc[:,'Quarter Number'] - 6 #Buat lag Quarter Number untuk join
    lapkeu_i_lag_6.drop(columns=['Kode'],inplace=True)
    lapkeu_i = lapkeu_i.merge(lapkeu_i_lag_6, how='outer', on='Quarter Number')  
    list_lapkeu_lag.append(lapkeu_i)
lapkeu_lag = pd.concat(list_lapkeu_lag).sort_values(by=['Kode','Quarter Number'])
lapkeu_lag = lapkeu_lag[~lapkeu_lag['Kode'].isna()]
lapkeu_lag = lapkeu_lag.reset_index(drop=True)

#Replace '' with Nan
lapkeu_lag.loc[:,'Aset Lancar Lag 2'] = lapkeu_lag['Aset Lancar Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tidak Lancar Lag 2'] = lapkeu_lag['Aset Tidak Lancar Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Lag 2'] = lapkeu_lag['Aset Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Jangka Pendek Lag 2'] = lapkeu_lag['Liabilitas Jangka Pendek Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Jangka Panjang Lag 2'] = lapkeu_lag['Liabilitas Jangka Panjang Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Lag 2'] = lapkeu_lag['Liabilitas Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Ekuitas Lag 2'] = lapkeu_lag['Ekuitas Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Dan Ekuitas Lag 2'] = lapkeu_lag['Liabilitas Dan Ekuitas Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Kas Dan Setara Kas Lag 2'] = lapkeu_lag['Kas Dan Setara Kas Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Premi Lag 2'] = lapkeu_lag['Piutang Premi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Reasuransi Lag 2'] = lapkeu_lag['Piutang Reasuransi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Penempatan Pada Bank Indonesia Dan Bank Lain Lag 2'] = lapkeu_lag['Penempatan Pada Bank Indonesia Dan Bank Lain Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Pinjaman Yang Diberikan Lag 2'] = lapkeu_lag['Pinjaman Yang Diberikan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tetap Lag 2'] = lapkeu_lag['Aset Tetap Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Sewa Pembiayaan Lag 2'] = lapkeu_lag['Piutang Sewa Pembiayaan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Pembiayaan Konsumen Lag 2'] = lapkeu_lag['Piutang Pembiayaan Konsumen Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Nasabah Lag 2'] = lapkeu_lag['Piutang Nasabah Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Lain-Lain Lag 2'] = lapkeu_lag['Piutang Lain-Lain Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Portofolio Efek Lag 2'] = lapkeu_lag['Portofolio Efek Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tak Berwujud Lag 2'] = lapkeu_lag['Aset Tak Berwujud Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Jangka Pendek Lag 2'] = lapkeu_lag['Investasi Jangka Pendek Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Reasuransi Lag 2'] = lapkeu_lag['Aset Reasuransi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Usaha Lag 2'] = lapkeu_lag['Piutang Usaha Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Premi Dan Reasuransi Lag 2'] = lapkeu_lag['Piutang Premi Dan Reasuransi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Jangka Panjang Lag 2'] = lapkeu_lag['Investasi Jangka Panjang Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Goodwill Lag 2'] = lapkeu_lag['Goodwill Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Pembiayaan Lag 2'] = lapkeu_lag['Piutang Pembiayaan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Obligasi Pemerintah Lag 2'] = lapkeu_lag['Obligasi Pemerintah Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Pada Entitas Asosiasi Lag 2'] = lapkeu_lag['Investasi Pada Entitas Asosiasi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Deposito Berjangka Lag 2'] = lapkeu_lag['Deposito Berjangka Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Efek Tersedia Untuk Dijual Lag 2'] = lapkeu_lag['Efek Tersedia Untuk Dijual Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Total Pendapatan Lag 2'] = lapkeu_lag['Total Pendapatan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Kotor Lag 2'] = lapkeu_lag['Laba Kotor Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Usaha Lag 2'] = lapkeu_lag['Laba Usaha Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Sebelum Pajak Lag 2'] = lapkeu_lag['Laba Sebelum Pajak Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Bersih Tahun Berjalan Lag 2'] = lapkeu_lag['Laba Bersih Tahun Berjalan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Jumlah Laba Komprehensif Lag 2'] = lapkeu_lag['Jumlah Laba Komprehensif Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Bersih Yang Dapat Diatribusikan Kepada Lag 2'] = lapkeu_lag['Laba Bersih Yang Dapat Diatribusikan Kepada Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Operasi Lag 2'] = lapkeu_lag['Arus Kas Dari Aktivitas Operasi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Investasi Lag 2'] = lapkeu_lag['Arus Kas Dari Aktivitas Investasi Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Pendanaan Lag 2'] = lapkeu_lag['Arus Kas Dari Aktivitas Pendanaan Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 2'] = lapkeu_lag['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 2'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Lancar Lag 6'] = lapkeu_lag['Aset Lancar Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tidak Lancar Lag 6'] = lapkeu_lag['Aset Tidak Lancar Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Lag 6'] = lapkeu_lag['Aset Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Jangka Pendek Lag 6'] = lapkeu_lag['Liabilitas Jangka Pendek Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Jangka Panjang Lag 6'] = lapkeu_lag['Liabilitas Jangka Panjang Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Lag 6'] = lapkeu_lag['Liabilitas Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Ekuitas Lag 6'] = lapkeu_lag['Ekuitas Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Liabilitas Dan Ekuitas Lag 6'] = lapkeu_lag['Liabilitas Dan Ekuitas Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Kas Dan Setara Kas Lag 6'] = lapkeu_lag['Kas Dan Setara Kas Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Premi Lag 6'] = lapkeu_lag['Piutang Premi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Reasuransi Lag 6'] = lapkeu_lag['Piutang Reasuransi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6'] = lapkeu_lag['Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Pinjaman Yang Diberikan Lag 6'] = lapkeu_lag['Pinjaman Yang Diberikan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tetap Lag 6'] = lapkeu_lag['Aset Tetap Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Sewa Pembiayaan Lag 6'] = lapkeu_lag['Piutang Sewa Pembiayaan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Pembiayaan Konsumen Lag 6'] = lapkeu_lag['Piutang Pembiayaan Konsumen Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Nasabah Lag 6'] = lapkeu_lag['Piutang Nasabah Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Lain-Lain Lag 6'] = lapkeu_lag['Piutang Lain-Lain Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Portofolio Efek Lag 6'] = lapkeu_lag['Portofolio Efek Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Tak Berwujud Lag 6'] = lapkeu_lag['Aset Tak Berwujud Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Jangka Pendek Lag 6'] = lapkeu_lag['Investasi Jangka Pendek Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Aset Reasuransi Lag 6'] = lapkeu_lag['Aset Reasuransi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Usaha Lag 6'] = lapkeu_lag['Piutang Usaha Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Premi Dan Reasuransi Lag 6'] = lapkeu_lag['Piutang Premi Dan Reasuransi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Jangka Panjang Lag 6'] = lapkeu_lag['Investasi Jangka Panjang Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Goodwill Lag 6'] = lapkeu_lag['Goodwill Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Piutang Pembiayaan Lag 6'] = lapkeu_lag['Piutang Pembiayaan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Obligasi Pemerintah Lag 6'] = lapkeu_lag['Obligasi Pemerintah Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Investasi Pada Entitas Asosiasi Lag 6'] = lapkeu_lag['Investasi Pada Entitas Asosiasi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Deposito Berjangka Lag 6'] = lapkeu_lag['Deposito Berjangka Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Efek Tersedia Untuk Dijual Lag 6'] = lapkeu_lag['Efek Tersedia Untuk Dijual Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Total Pendapatan Lag 6'] = lapkeu_lag['Total Pendapatan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Kotor Lag 6'] = lapkeu_lag['Laba Kotor Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Usaha Lag 6'] = lapkeu_lag['Laba Usaha Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Sebelum Pajak Lag 6'] = lapkeu_lag['Laba Sebelum Pajak Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Bersih Tahun Berjalan Lag 6'] = lapkeu_lag['Laba Bersih Tahun Berjalan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Jumlah Laba Komprehensif Lag 6'] = lapkeu_lag['Jumlah Laba Komprehensif Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6'] = lapkeu_lag['Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Operasi Lag 6'] = lapkeu_lag['Arus Kas Dari Aktivitas Operasi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Investasi Lag 6'] = lapkeu_lag['Arus Kas Dari Aktivitas Investasi Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Pendanaan Lag 6'] = lapkeu_lag['Arus Kas Dari Aktivitas Pendanaan Lag 6'].replace('',np.nan)
lapkeu_lag.loc[:,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'] = lapkeu_lag['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'].replace('',np.nan)

#Hitung growth
lapkeu_lag.loc[:,'Aset Lancar Growth YoY'] = (lapkeu_lag['Aset Lancar Lag 2'].astype('float64')-lapkeu_lag['Aset Lancar Lag 6'].astype('float64'))/lapkeu_lag['Aset Lancar Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Aset Tidak Lancar Growth YoY'] = (lapkeu_lag['Aset Tidak Lancar Lag 2'].astype('float64')-lapkeu_lag['Aset Tidak Lancar Lag 6'].astype('float64'))/lapkeu_lag['Aset Tidak Lancar Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Aset Growth YoY'] = (lapkeu_lag['Aset Lag 2'].astype('float64')-lapkeu_lag['Aset Lag 6'].astype('float64'))/lapkeu_lag['Aset Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Liabilitas Jangka Pendek Growth YoY'] = (lapkeu_lag['Liabilitas Jangka Pendek Lag 2'].astype('float64')-lapkeu_lag['Liabilitas Jangka Pendek Lag 6'].astype('float64'))/lapkeu_lag['Liabilitas Jangka Pendek Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Liabilitas Jangka Panjang Growth YoY'] = (lapkeu_lag['Liabilitas Jangka Panjang Lag 2'].astype('float64')-lapkeu_lag['Liabilitas Jangka Panjang Lag 6'].astype('float64'))/lapkeu_lag['Liabilitas Jangka Panjang Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Liabilitas Growth YoY'] = (lapkeu_lag['Liabilitas Lag 2'].astype('float64')-lapkeu_lag['Liabilitas Lag 6'].astype('float64'))/lapkeu_lag['Liabilitas Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Ekuitas Growth YoY'] = (lapkeu_lag['Ekuitas Lag 2'].astype('float64')-lapkeu_lag['Ekuitas Lag 6'].astype('float64'))/lapkeu_lag['Ekuitas Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Liabilitas Dan Ekuitas Growth YoY'] = (lapkeu_lag['Liabilitas Dan Ekuitas Lag 2'].astype('float64')-lapkeu_lag['Liabilitas Dan Ekuitas Lag 6'].astype('float64'))/lapkeu_lag['Liabilitas Dan Ekuitas Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Kas Dan Setara Kas Growth YoY'] = (lapkeu_lag['Kas Dan Setara Kas Lag 2'].astype('float64')-lapkeu_lag['Kas Dan Setara Kas Lag 6'].astype('float64'))/lapkeu_lag['Kas Dan Setara Kas Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Premi Growth YoY'] = (lapkeu_lag['Piutang Premi Lag 2'].astype('float64')-lapkeu_lag['Piutang Premi Lag 6'].astype('float64'))/lapkeu_lag['Piutang Premi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Reasuransi Growth YoY'] = (lapkeu_lag['Piutang Reasuransi Lag 2'].astype('float64')-lapkeu_lag['Piutang Reasuransi Lag 6'].astype('float64'))/lapkeu_lag['Piutang Reasuransi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Penempatan Pada Bank Indonesia Dan Bank Lain Growth YoY'] = (lapkeu_lag['Penempatan Pada Bank Indonesia Dan Bank Lain Lag 2'].astype('float64')-lapkeu_lag['Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6'].astype('float64'))/lapkeu_lag['Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Pinjaman Yang Diberikan Growth YoY'] = (lapkeu_lag['Pinjaman Yang Diberikan Lag 2'].astype('float64')-lapkeu_lag['Pinjaman Yang Diberikan Lag 6'].astype('float64'))/lapkeu_lag['Pinjaman Yang Diberikan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Aset Tetap Growth YoY'] = (lapkeu_lag['Aset Tetap Lag 2'].astype('float64')-lapkeu_lag['Aset Tetap Lag 6'].astype('float64'))/lapkeu_lag['Aset Tetap Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Sewa Pembiayaan Growth YoY'] = (lapkeu_lag['Piutang Sewa Pembiayaan Lag 2'].astype('float64')-lapkeu_lag['Piutang Sewa Pembiayaan Lag 6'].astype('float64'))/lapkeu_lag['Piutang Sewa Pembiayaan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Pembiayaan Konsumen Growth YoY'] = (lapkeu_lag['Piutang Pembiayaan Konsumen Lag 2'].astype('float64')-lapkeu_lag['Piutang Pembiayaan Konsumen Lag 6'].astype('float64'))/lapkeu_lag['Piutang Pembiayaan Konsumen Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Nasabah Growth YoY'] = (lapkeu_lag['Piutang Nasabah Lag 2'].astype('float64')-lapkeu_lag['Piutang Nasabah Lag 6'].astype('float64'))/lapkeu_lag['Piutang Nasabah Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Lain-Lain Growth YoY'] = (lapkeu_lag['Piutang Lain-Lain Lag 2'].astype('float64')-lapkeu_lag['Piutang Lain-Lain Lag 6'].astype('float64'))/lapkeu_lag['Piutang Lain-Lain Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Portofolio Efek Growth YoY'] = (lapkeu_lag['Portofolio Efek Lag 2'].astype('float64')-lapkeu_lag['Portofolio Efek Lag 6'].astype('float64'))/lapkeu_lag['Portofolio Efek Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Aset Tak Berwujud Growth YoY'] = (lapkeu_lag['Aset Tak Berwujud Lag 2'].astype('float64')-lapkeu_lag['Aset Tak Berwujud Lag 6'].astype('float64'))/lapkeu_lag['Aset Tak Berwujud Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Investasi Jangka Pendek Growth YoY'] = (lapkeu_lag['Investasi Jangka Pendek Lag 2'].astype('float64')-lapkeu_lag['Investasi Jangka Pendek Lag 6'].astype('float64'))/lapkeu_lag['Investasi Jangka Pendek Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Aset Reasuransi Growth YoY'] = (lapkeu_lag['Aset Reasuransi Lag 2'].astype('float64')-lapkeu_lag['Aset Reasuransi Lag 6'].astype('float64'))/lapkeu_lag['Aset Reasuransi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Usaha Growth YoY'] = (lapkeu_lag['Piutang Usaha Lag 2'].astype('float64')-lapkeu_lag['Piutang Usaha Lag 6'].astype('float64'))/lapkeu_lag['Piutang Usaha Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Premi Dan Reasuransi Growth YoY'] = (lapkeu_lag['Piutang Premi Dan Reasuransi Lag 2'].astype('float64')-lapkeu_lag['Piutang Premi Dan Reasuransi Lag 6'].astype('float64'))/lapkeu_lag['Piutang Premi Dan Reasuransi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Investasi Jangka Panjang Growth YoY'] = (lapkeu_lag['Investasi Jangka Panjang Lag 2'].astype('float64')-lapkeu_lag['Investasi Jangka Panjang Lag 6'].astype('float64'))/lapkeu_lag['Investasi Jangka Panjang Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Goodwill Growth YoY'] = (lapkeu_lag['Goodwill Lag 2'].astype('float64')-lapkeu_lag['Goodwill Lag 6'].astype('float64'))/lapkeu_lag['Goodwill Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Piutang Pembiayaan Growth YoY'] = (lapkeu_lag['Piutang Pembiayaan Lag 2'].astype('float64')-lapkeu_lag['Piutang Pembiayaan Lag 6'].astype('float64'))/lapkeu_lag['Piutang Pembiayaan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Obligasi Pemerintah Growth YoY'] = (lapkeu_lag['Obligasi Pemerintah Lag 2'].astype('float64')-lapkeu_lag['Obligasi Pemerintah Lag 6'].astype('float64'))/lapkeu_lag['Obligasi Pemerintah Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Investasi Pada Entitas Asosiasi Growth YoY'] = (lapkeu_lag['Investasi Pada Entitas Asosiasi Lag 2'].astype('float64')-lapkeu_lag['Investasi Pada Entitas Asosiasi Lag 6'].astype('float64'))/lapkeu_lag['Investasi Pada Entitas Asosiasi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Deposito Berjangka Growth YoY'] = (lapkeu_lag['Deposito Berjangka Lag 2'].astype('float64')-lapkeu_lag['Deposito Berjangka Lag 6'].astype('float64'))/lapkeu_lag['Deposito Berjangka Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Efek Tersedia Untuk Dijual Growth YoY'] = (lapkeu_lag['Efek Tersedia Untuk Dijual Lag 2'].astype('float64')-lapkeu_lag['Efek Tersedia Untuk Dijual Lag 6'].astype('float64'))/lapkeu_lag['Efek Tersedia Untuk Dijual Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Total Pendapatan Growth YoY'] = (lapkeu_lag['Total Pendapatan Lag 2'].astype('float64')-lapkeu_lag['Total Pendapatan Lag 6'].astype('float64'))/lapkeu_lag['Total Pendapatan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Laba Kotor Growth YoY'] = (lapkeu_lag['Laba Kotor Lag 2'].astype('float64')-lapkeu_lag['Laba Kotor Lag 6'].astype('float64'))/lapkeu_lag['Laba Kotor Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Laba Usaha Growth YoY'] = (lapkeu_lag['Laba Usaha Lag 2'].astype('float64')-lapkeu_lag['Laba Usaha Lag 6'].astype('float64'))/lapkeu_lag['Laba Usaha Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Laba Sebelum Pajak Growth YoY'] = (lapkeu_lag['Laba Sebelum Pajak Lag 2'].astype('float64')-lapkeu_lag['Laba Sebelum Pajak Lag 6'].astype('float64'))/lapkeu_lag['Laba Sebelum Pajak Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Laba Bersih Tahun Berjalan Growth YoY'] = (lapkeu_lag['Laba Bersih Tahun Berjalan Lag 2'].astype('float64')-lapkeu_lag['Laba Bersih Tahun Berjalan Lag 6'].astype('float64'))/lapkeu_lag['Laba Bersih Tahun Berjalan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Jumlah Laba Komprehensif Growth YoY'] = (lapkeu_lag['Jumlah Laba Komprehensif Lag 2'].astype('float64')-lapkeu_lag['Jumlah Laba Komprehensif Lag 6'].astype('float64'))/lapkeu_lag['Jumlah Laba Komprehensif Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Laba Bersih Yang Dapat Diatribusikan Kepada Growth YoY'] = (lapkeu_lag['Laba Bersih Yang Dapat Diatribusikan Kepada Lag 2'].astype('float64')-lapkeu_lag['Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6'].astype('float64'))/lapkeu_lag['Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Operasi Growth YoY'] = (lapkeu_lag['Arus Kas Dari Aktivitas Operasi Lag 2'].astype('float64')-lapkeu_lag['Arus Kas Dari Aktivitas Operasi Lag 6'].astype('float64'))/lapkeu_lag['Arus Kas Dari Aktivitas Operasi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Investasi Growth YoY'] = (lapkeu_lag['Arus Kas Dari Aktivitas Investasi Lag 2'].astype('float64')-lapkeu_lag['Arus Kas Dari Aktivitas Investasi Lag 6'].astype('float64'))/lapkeu_lag['Arus Kas Dari Aktivitas Investasi Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Arus Kas Dari Aktivitas Pendanaan Growth YoY'] = (lapkeu_lag['Arus Kas Dari Aktivitas Pendanaan Lag 2'].astype('float64')-lapkeu_lag['Arus Kas Dari Aktivitas Pendanaan Lag 6'].astype('float64'))/lapkeu_lag['Arus Kas Dari Aktivitas Pendanaan Lag 6'].astype('float64')
lapkeu_lag.loc[:,'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Growth YoY'] = (lapkeu_lag['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 2'].astype('float64')-lapkeu_lag['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'].astype('float64'))/lapkeu_lag['Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'].astype('float64')

#Drop kolom lag 6
lapkeu = lapkeu_lag.drop(columns=['Aset Lancar Lag 6','Aset Tidak Lancar Lag 6','Aset Lag 6','Liabilitas Jangka Pendek Lag 6','Liabilitas Jangka Panjang Lag 6','Liabilitas Lag 6','Ekuitas Lag 6','Liabilitas Dan Ekuitas Lag 6',
                                  'Kas Dan Setara Kas Lag 6','Piutang Premi Lag 6','Piutang Reasuransi Lag 6','Penempatan Pada Bank Indonesia Dan Bank Lain Lag 6','Pinjaman Yang Diberikan Lag 6','Aset Tetap Lag 6',
                                  'Piutang Sewa Pembiayaan Lag 6','Piutang Pembiayaan Konsumen Lag 6','Piutang Nasabah Lag 6','Piutang Lain-Lain Lag 6','Portofolio Efek Lag 6','Aset Tak Berwujud Lag 6',
                                  'Investasi Jangka Pendek Lag 6','Aset Reasuransi Lag 6','Piutang Usaha Lag 6','Piutang Premi Dan Reasuransi Lag 6','Investasi Jangka Panjang Lag 6','Goodwill Lag 6','Piutang Pembiayaan Lag 6',
                                  'Obligasi Pemerintah Lag 6','Investasi Pada Entitas Asosiasi Lag 6','Deposito Berjangka Lag 6','Efek Tersedia Untuk Dijual Lag 6','Total Pendapatan Lag 6','Laba Kotor Lag 6',
                                  'Laba Usaha Lag 6','Laba Sebelum Pajak Lag 6','Laba Bersih Tahun Berjalan Lag 6','Jumlah Laba Komprehensif Lag 6','Laba Bersih Yang Dapat Diatribusikan Kepada Lag 6',
                                  'Arus Kas Dari Aktivitas Operasi Lag 6','Arus Kas Dari Aktivitas Investasi Lag 6','Arus Kas Dari Aktivitas Pendanaan Lag 6','Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 6'])


/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/11474873.py:135: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lapkeu_lag.loc[:,'Total Pendapatan Lag 2'] = lapkeu_lag['Total Pendapatan Lag 2'].replace('',np.nan)
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/11474873.py:136: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lapkeu_lag.loc[:,'Laba Kotor Lag 2'] = lapkeu_lag['Laba Kotor Lag 2'].replace('',np.nan)
/var/folders/z9/y0mn7sxd60n70knn0w02k8h00000gn/T/ipykernel_46662/11474873.py:137: FutureWa

In [16]:
#Buat kolom date
list_lapkeu = []
for i in set(list(lapkeu['Quarter Year'])):
    lapkeu_i = lapkeu[lapkeu['Quarter Year']==i]
    date_from = lapkeu_i['Date From'].unique()[0]
    date_to = lapkeu_i['Date To'].unique()[0]
    list_lapkeu_k = []
    for k in pd.bdate_range(start=date_from,end=date_to):
        lapkeu_k = lapkeu_i.copy()
        lapkeu_k.loc[:,'Date'] = k
        list_lapkeu_k.append(lapkeu_k)
    lapkeu_k = pd.concat(list_lapkeu_k)
    list_lapkeu.append(lapkeu_k)
lapkeu_all_date = pd.concat(list_lapkeu).reset_index(drop=True)
lapkeu_all_date = lapkeu_all_date.sort_values(by=['Date','Kode']).reset_index(drop=True)

In [17]:
#Drop kolom lag 0 supaya semua data fundamental sudah lag 2 quarter
lapkeu_all_date_lag_2 = lapkeu_all_date.drop(columns=['Aset Lancar','Aset Tidak Lancar','Aset','Liabilitas Jangka Pendek','Liabilitas Jangka Panjang','Liabilitas','Ekuitas','Liabilitas Dan Ekuitas',
                                  'Kas Dan Setara Kas','Piutang Premi','Piutang Reasuransi','Penempatan Pada Bank Indonesia Dan Bank Lain','Pinjaman Yang Diberikan','Aset Tetap',
                                  'Piutang Sewa Pembiayaan','Piutang Pembiayaan Konsumen','Piutang Nasabah','Piutang Lain-Lain','Portofolio Efek','Aset Tak Berwujud',
                                  'Investasi Jangka Pendek','Aset Reasuransi','Piutang Usaha','Piutang Premi Dan Reasuransi','Investasi Jangka Panjang','Goodwill','Piutang Pembiayaan',
                                  'Obligasi Pemerintah','Investasi Pada Entitas Asosiasi','Deposito Berjangka','Efek Tersedia Untuk Dijual','Total Pendapatan','Laba Kotor',
                                  'Laba Usaha','Laba Sebelum Pajak','Laba Bersih Tahun Berjalan','Jumlah Laba Komprehensif','Laba Bersih Yang Dapat Diatribusikan Kepada',
                                  'Arus Kas Dari Aktivitas Operasi','Arus Kas Dari Aktivitas Investasi','Arus Kas Dari Aktivitas Pendanaan','Kenaikan (Penurunan) Bersih Kas Dan Setara Kas'])

In [18]:
selected_columns = ['portid','Quarter Year','portdate','opening','high','low','closing','volume','valuess','foreignsell','foreignbuy',
                    'Saham Beredar','Aset Lancar Lag 2','Aset Tidak Lancar Lag 2','Aset Lag 2','Liabilitas Jangka Pendek Lag 2',
                    'Liabilitas Jangka Panjang Lag 2','Liabilitas Lag 2','Ekuitas Lag 2','Liabilitas Dan Ekuitas Lag 2','Kas Dan Setara Kas Lag 2',
                    'Piutang Premi Lag 2','Piutang Reasuransi Lag 2','Penempatan Pada Bank Indonesia Dan Bank Lain Lag 2',
                    'Pinjaman Yang Diberikan Lag 2','Aset Tetap Lag 2','Piutang Sewa Pembiayaan Lag 2','Piutang Pembiayaan Konsumen Lag 2',
                    'Piutang Nasabah Lag 2','Piutang Lain-Lain Lag 2','Portofolio Efek Lag 2','Aset Tak Berwujud Lag 2',
                    'Investasi Jangka Pendek Lag 2','Aset Reasuransi Lag 2','Piutang Usaha Lag 2','Piutang Premi Dan Reasuransi Lag 2',
                    'Investasi Jangka Panjang Lag 2','Goodwill Lag 2','Piutang Pembiayaan Lag 2','Obligasi Pemerintah Lag 2',
                    'Investasi Pada Entitas Asosiasi Lag 2','Deposito Berjangka Lag 2','Efek Tersedia Untuk Dijual Lag 2',
                    'Total Pendapatan Lag 2','Laba Kotor Lag 2','Laba Usaha Lag 2','Laba Sebelum Pajak Lag 2','Laba Bersih Tahun Berjalan Lag 2',
                    'Jumlah Laba Komprehensif Lag 2','Laba Bersih Yang Dapat Diatribusikan Kepada Lag 2','Arus Kas Dari Aktivitas Operasi Lag 2',
                    'Arus Kas Dari Aktivitas Investasi Lag 2','Arus Kas Dari Aktivitas Pendanaan Lag 2','Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Lag 2',
                    'Aset Lancar Growth YoY','Aset Tidak Lancar Growth YoY','Aset Growth YoY','Liabilitas Jangka Pendek Growth YoY',
                    'Liabilitas Jangka Panjang Growth YoY','Liabilitas Growth YoY','Ekuitas Growth YoY','Liabilitas Dan Ekuitas Growth YoY','Kas Dan Setara Kas Growth YoY',
                    'Piutang Premi Growth YoY','Piutang Reasuransi Growth YoY','Penempatan Pada Bank Indonesia Dan Bank Lain Growth YoY',
                    'Pinjaman Yang Diberikan Growth YoY','Aset Tetap Growth YoY','Piutang Sewa Pembiayaan Growth YoY','Piutang Pembiayaan Konsumen Growth YoY',
                    'Piutang Nasabah Growth YoY','Piutang Lain-Lain Growth YoY','Portofolio Efek Growth YoY','Aset Tak Berwujud Growth YoY',
                    'Investasi Jangka Pendek Growth YoY','Aset Reasuransi Growth YoY','Piutang Usaha Growth YoY','Piutang Premi Dan Reasuransi Growth YoY',
                    'Investasi Jangka Panjang Growth YoY','Goodwill Growth YoY','Piutang Pembiayaan Growth YoY','Obligasi Pemerintah Growth YoY',
                    'Investasi Pada Entitas Asosiasi Growth YoY','Deposito Berjangka Growth YoY','Efek Tersedia Untuk Dijual Growth YoY',
                    'Total Pendapatan Growth YoY','Laba Kotor Growth YoY','Laba Usaha Growth YoY','Laba Sebelum Pajak Growth YoY',
                    'Laba Bersih Tahun Berjalan Growth YoY','Jumlah Laba Komprehensif Growth YoY','Laba Bersih Yang Dapat Diatribusikan Kepada Growth YoY',
                    'Arus Kas Dari Aktivitas Operasi Growth YoY','Arus Kas Dari Aktivitas Investasi Growth YoY','Arus Kas Dari Aktivitas Pendanaan Growth YoY',
                    'Kenaikan (Penurunan) Bersih Kas Dan Setara Kas Growth YoY']

In [19]:
#Merge data dan ubah nama kolom
lapkeu_all_date_lag_2['Kode + Date'] = lapkeu_all_date_lag_2['Kode'].astype('str') + ' ' + lapkeu_all_date_lag_2['Date'].astype('str')
df_all['Kode + Date'] = df_all['portid'].astype('str') + ' ' + df_all['portdate'].astype('str')
database = df_all.merge(lapkeu_all_date_lag_2.drop(['Kode','Date'],axis=1), how='inner', on='Kode + Date')
database = database.drop('Kode + Date',axis=1)
database = database.loc[:,selected_columns] #pilih hanya kolom tertentu
database.columns = database.columns.str.replace(' Lag 2','') #database sudah pakai data fundamental lag 2
database.columns = database.columns.str.replace('portid','Kode')
database.columns = database.columns.str.replace('portdate','Date')
database.columns = database.columns.str.replace('opening','Open Price')
database.columns = database.columns.str.replace('high','High Price')
database.columns = database.columns.str.replace('low','Low Price')
database.columns = database.columns.str.replace('closing','Close Price')
database.columns = database.columns.str.replace('valuess','Transaction Value')

In [20]:
#Ganti saham beredar menjadi saham beredar yang terakhir, untuk perkalian market cap
stocks = pd.DataFrame(database['Kode'].sort_values().unique(),columns=['Kode'])
out_shares_dic = {}
for i in stocks['Kode']:
    d = database[database['Kode']==i]['Saham Beredar'].reset_index(drop=True)
    try:
        out_shares_dic[i] = d.iloc[d.last_valid_index()]
    except TypeError:
        out_shares_dic[i] = np.nan
out_shares = pd.DataFrame(out_shares_dic.items(),columns=['Kode','Last Outstanding Shares'])
database = database.merge(out_shares,how='inner', on='Kode')
database = database.drop(columns='Saham Beredar')

In [21]:
#Buat market cap, log market cap, dan estimated transaction value
database['Market Cap'] = database['Close Price']*database['Last Outstanding Shares']
database['Log Market Cap'] = np.log10(database['Market Cap'])
database['Estimated Transaction Value'] = ((database['High Price']+database['Low Price'])/2)*database['volume']

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
#Buat kolom momentum untuk berbagai periode. Asumsi 1 bulan = 20 hari
database_mom = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Momentum 1 Day'] = database_i['Close Price'].pct_change(periods=1)
    database_i['Momentum 3 Day'] = database_i['Close Price'].pct_change(periods=3)
    database_i['Momentum 5 Day'] = database_i['Close Price'].pct_change(periods=5)
    database_i['Momentum 10 Day'] = database_i['Close Price'].pct_change(periods=10)
    database_i['Momentum 1 Month'] = database_i['Close Price'].pct_change(periods=20)
    database_i['Momentum 2 Month'] = database_i['Close Price'].pct_change(periods=40)
    database_i['Momentum 3 Month'] = database_i['Close Price'].pct_change(periods=60)
    database_i['Momentum 6 Month'] = database_i['Close Price'].pct_change(periods=120)
    database_i['Momentum 1 Year'] = database_i['Close Price'].pct_change(periods=240)
    database_i['Momentum 12-2'] = database_i['Close Price'].pct_change(periods=220).shift(20)
    database_i['Total Return 1 Day'] = database_i['Close Price'].pct_change(periods=1).shift(-1)
    database_i['Total Return 3 Day'] = database_i['Close Price'].pct_change(periods=3).shift(-3)
    database_i['Total Return 5 Day'] = database_i['Close Price'].pct_change(periods=5).shift(-5)
    database_i['Total Return 10 Day'] = database_i['Close Price'].pct_change(periods=10).shift(-10)
    database_i['Total Return 1 Month'] = database_i['Close Price'].pct_change(periods=20).shift(-20)
    database_i['Total Return 2 Month'] = database_i['Close Price'].pct_change(periods=40).shift(-40)
    database_i['Total Return 3 Month'] = database_i['Close Price'].pct_change(periods=60).shift(-60)
    database_i['Total Return 6 Month'] = database_i['Close Price'].pct_change(periods=120).shift(-120)
    database_i['Change in Estimated Transaction Value 1 Month'] = database_i['Estimated Transaction Value'].pct_change(periods=20)
    database_mom.append(database_i)
database = pd.concat(database_mom).sort_values(by=['Kode','Date'])

In [23]:
#Buat average Market Cap 1,3, dan 6 bulan sebelum
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Market Cap 6 Month'] = database_i['Market Cap'].rolling(120).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Market Cap 3 Month'] = database_i['Market Cap'].rolling(60).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Market Cap 1 Month'] = database_i['Market Cap'].rolling(20).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Estimated Transaction Value 1 Month'] = database_i['Estimated Transaction Value'].rolling(20).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

#Buat Average Volatility 1 bulan 
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Volatility 1 Month'] = database_i['Momentum 1 Day'].rolling(20).std()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

In [31]:
database[database['Date'] == '2025-01-07'][['Kode', 'Date', 'Aset',
                                            'Liabilitas Jangka Pendek',
                                            'Liabilitas Jangka Panjang',
                                            'Liabilitas',
                                            'Ekuitas',]]

,Kode,Date,Aset,Liabilitas Jangka Pendek,Liabilitas Jangka Panjang,Liabilitas,Ekuitas
2304867,AADI,2025-01-07,8.906922e+13,2.491662e+13,1.954506e+13,4.446168e+13,4.460755e+13
2304387,AALI,2025-01-07,2.965048e+13,4.459708e+12,2.442690e+12,6.902398e+12,2.274809e+13
2304540,ABBA,2025-01-07,2.328550e+11,2.133263e+11,1.302161e+11,3.435424e+11,-1.106874e+11
2304537,ABDA,2025-01-07,2.657877e+12,NaN,NaN,1.138494e+12,1.519383e+12
2304247,ABMM,2025-01-07,3.462571e+13,1.047356e+13,1.124297e+13,2.171652e+13,1.290918e+13
...,...,...,...,...,...,...,...
2304539,ZATA,2025-01-07,6.734727e+11,4.407679e+10,1.026615e+11,1.467383e+11,5.267345e+11
2304897,ZBRA,2025-01-07,3.006691e+12,1.703924e+12,5.352645e+10,1.757451e+12,9.674301e+11
2304924,ZINC,2025-01-07,2.548548e+12,4.002123e+11,1.419771e+12,1.819983e+12,7.285649e+11
2304358,ZONE,2025-01-07,6.617367e+11,1.398564e+11,1.298260e+11,2.696824e+11,3.920542e+11


In [32]:
#Buat PBV
database['PBV'] = database['Market Cap']/database['Ekuitas']
database['Average PBV 1 Month'] = database['Average Market Cap 1 Month']/database['Ekuitas']
database['Average PBV 3 Month'] = database['Average Market Cap 3 Month']/database['Ekuitas']
database['Average PBV 6 Month'] = database['Average Market Cap 6 Month']/database['Ekuitas']

#Buat Price/Sales
database['Price/Sales'] = database['Market Cap']/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Average Price/Sales 1 Month'] = database['Average Market Cap 1 Month']/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Average Price/Sales 3 Month'] = database['Average Market Cap 3 Month']/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Average Price/Sales 6 Month'] = database['Average Market Cap 6 Month']/database['Total Pendapatan'].replace('',np.nan).astype(float)

#Buat PE Ratio
database['PE Ratio'] = database['Market Cap']/database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)
database['Average PE Ratio 1 Month'] = database['Average Market Cap 1 Month']/database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)

#Buat Price/Operating Cash Flow
database['Price/Operating Cash Flow'] = database['Market Cap']/database['Arus Kas Dari Aktivitas Operasi'].replace('',np.nan).astype(float)
database['Average Price/Operating Cash Flow 1 Month'] = database['Average Market Cap 1 Month']/database['Arus Kas Dari Aktivitas Operasi'].replace('',np.nan).astype(float)

#Buat ROE
database['ROE'] = database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)/database['Ekuitas']

#Buat Gross, Operating, dan Net Profit Margin
database['Net Profit Margin'] = database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Gross Profit Margin'] = database['Laba Kotor'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Operating Profit Margin'] = database['Laba Usaha'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)

#Buat Debt Ratio
database['Debt Ratio'] = database['Liabilitas']/database['Aset']

#Tambahkan IHSG
ihsg = pd.read_excel('IHSG.xls',skiprows=[0])
ihsg = ihsg[['Tanggal','Nilai']]
ihsg = ihsg.rename(columns={'Tanggal':'Date','Nilai':'IHSG'})
ihsg['IHSG 1 Day Return'] = ihsg['IHSG'].pct_change()
ihsg = ihsg.dropna()
ihsg['Date'] = ihsg['Date'].astype('str')
ihsg = ihsg.reset_index(drop=True)
database = database.merge(ihsg, how='right', on='Date')

#Tambahkan Risk-Free Rate
rf = pd.read_excel('Suku Bunga Deposito 1 Bulan (LPS).xls',skiprows=[0])
rf['Rf'] = ((rf['Nilai']*0.01)+1)**(1/12)-1
rf = rf.rename(columns={'Tanggal':'Date'})
rf = rf.drop(columns=['Indeks','Nilai'])
rf['Month'] = [x.month for x in rf['Date']]
rf['Year'] = [x.year for x in rf['Date']]
rf = rf.reset_index(drop=True)
for i in range(len(rf)):
    if rf.loc[i,'Month'] == 1:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-01-31'
    elif rf.loc[i,'Month'] == 2:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-02-28'
    elif rf.loc[i,'Month'] == 3:
        year = lapkeu_sb.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-03-31'
    elif rf.loc[i,'Month'] == 4:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-04-30'
    elif rf.loc[i,'Month'] == 5:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-05-31'
    elif rf.loc[i,'Month'] == 6:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-06-30'
    elif rf.loc[i,'Month'] == 7:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-07-31'
    elif rf.loc[i,'Month'] == 8:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-08-31'
    elif rf.loc[i,'Month'] == 9:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-09-30'
    elif rf.loc[i,'Month'] == 10:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-10-31'
    elif rf.loc[i,'Month'] == 11:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-11-30'
    elif rf.loc[i,'Month'] == 12:
        year = rf.loc[i,'Year']
        rf.loc[i,'Date From'] = rf.loc[i,'Date']
        rf.loc[i,'Date To'] = f'{year}-12-31'
    else:
        rf.loc[i,'Date From'] = '' 
        rf.loc[i,'Date To'] = ''
list_rf= []
for i in set(list(rf['Date'])):
    rf_i = rf[rf['Date']==i]
    date_from = rf_i['Date From'].unique()[0]
    date_to = rf_i['Date To'].unique()[0]
    list_rf_k = []
    for k in pd.bdate_range(start=date_from,end=date_to):
        rf_k = rf_i.copy()
        rf_k.loc[:,'Date'] = k
        list_rf_k.append(rf_k)
    try:
        rf_k = pd.concat(list_rf_k)
        list_rf.append(rf_k)
    except ValueError:
        pass
rf_all_date = pd.concat(list_rf)
rf_all_date = rf_all_date.sort_values(by=['Date','Rf'])
rf_all_date = rf_all_date.drop_duplicates(subset='Date', keep='first').reset_index(drop=True)
rf_all_date = rf_all_date[['Date','Rf']]
rf_all_date['Date'] = rf_all_date['Date'].astype('str')
database = database.merge(rf_all_date, how='right', on='Date')
database = database.dropna(subset=["Kode"])
database = database.reset_index(drop=True)


In [21]:
#Tambahkan beta

#list_saham = list(idx30_stocks['Kode'].unique()) # biar ga lama
#beta_1_code_list = []
#beta_1_date_list = []
#beta_1_list = []
#beta_period = 240 # 1 year
#for i in list_saham:
#    print(i)
#    single_stock = df_all_funda_lag_2[df_all_funda_lag_2['Kode']==i]
#    single_stock = single_stock.reset_index(drop=True)
#    date_j_list = list(single_stock['Date'])
#    for j in date_j_list:
#        j_index = date_j_list.index(j)
#        print(j)
#        if j_index >= (beta_period):
#            data_whole_p = single_stock.iloc[j_index-beta_period:j_index]
#            data_whole_p = data_whole_p[['Kode','Date','Backward Close-Close Return 1','Backward Close-Close Return 3','IHSG Return','IHSG 3 days Return']]
#            vol_i = data_whole_p['Backward Close-Close Return 1'].std()
#            vol_m = data_whole_p['IHSG Return'].std()
#            data_corr = data_whole_p[['Backward Close-Close Return 3','IHSG 3 days Return']]
#            correl = data_corr.corr().iloc[0][1]
#            beta_1_year = correl*(vol_i/vol_m)
#            beta_1_code_list.append(i)
#            beta_1_date_list.append(j)
#            beta_1_list.append(beta_1_year)
#beta_1_params = pd.DataFrame({'Date':beta_1_date_list,
#                              'Kode':beta_1_code_list,
#                              'Beta 1 Year':beta_1_list})
#beta_1_params = beta_1_params.dropna()
#beta_1_params = beta_1_params.reset_index(drop=True)

In [33]:
#Tambah Daily Frekuensi (ambil dari Stock Summary BEI, cek frekuensi.ipynb)
freq_1 = pd.read_csv('Data Frekuensi 1.csv')
freq_2 = pd.read_csv('Data Frekuensi 2.csv')
freq = pd.concat([freq_1,freq_2])
freq['Kode + Date'] = freq['Kode'] + freq['Date'].astype('str')
database['Kode + Date'] = database['Kode'] + database['Date'].astype('str')
database = database.merge(freq[['Frequency','Kode + Date']],on='Kode + Date',how='outer')
database = database.drop(columns='Kode + Date')


In [34]:
#Buat average Transaction Value dan Frekuensi
list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Transaction Value 1 Month'] = database_i['Transaction Value'].rolling(20).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Transaction Value 3 Month'] = database_i['Transaction Value'].rolling(60).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Transaction Value 6 Month'] = database_i['Transaction Value'].rolling(120).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Transaction Value 1 Year'] = database_i['Transaction Value'].rolling(240).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])

list_database = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Average Frequency 1 Month'] = database_i['Frequency'].rolling(20).mean()
    list_database.append(database_i)
database = pd.concat(list_database).sort_values(by=['Kode','Date'])
database = database.reset_index(drop=True)

In [35]:
size = len(database)
database = database.reset_index(drop=True)
database_1 = database.iloc[:int(size/4)]
database_2 = database.iloc[int(size/4):int(size/2)]
database_3 = database.iloc[int(size/2):int(size*(3/4))]
database_4 = database.iloc[int(size*(3/4)):]


In [ ]:
database_1.to_csv('/Users/arielgraham/Documents/Infovesta Tasks/Database/Database Part 1.csv')
database_2.to_csv('/Users/arielgraham/Documents/Infovesta Tasks/Database/Database Part 2.csv')
database_3.to_csv('/Users/arielgraham/Documents/Infovesta Tasks/Database/Database Part 3.csv')
database_4.to_csv('/Users/arielgraham/Documents/Infovesta Tasks/Database/Database Part 4.csv')


: 